**Question**: Can a predictive model be built using leaf morphology classifications that may indicate that a particular genotype was cultivated in a “drought” or “control” condition?

In [13]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
HOME_DIR = Path().cwd().parent

SHEET_DATA = pd.read_excel(f'{HOME_DIR}/data.xlsx')
SHEET_DATA.head()

len(SHEET_DATA)

1672

In [17]:
LIST_OF_CLASSES = ['class1', 'class2', 'class3']
enc = OneHotEncoder()
pd.DataFrame(enc.fit_transform(SHEET_DATA))

,0
0,"(0, 1545)\t1.0\n (0, 1673)\t1.0\n (0, 1675..."
1,"(0, 610)\t1.0\n (0, 1672)\t1.0\n (0, 1675)..."
2,"(0, 1618)\t1.0\n (0, 1672)\t1.0\n (0, 1676..."
3,"(0, 960)\t1.0\n (0, 1672)\t1.0\n (0, 1676)..."
4,"(0, 1374)\t1.0\n (0, 1672)\t1.0\n (0, 1675..."
...,...
1667,"(0, 967)\t1.0\n (0, 1674)\t1.0\n (0, 1682)..."
1668,"(0, 169)\t1.0\n (0, 1672)\t1.0\n (0, 1675)..."
1669,"(0, 1430)\t1.0\n (0, 1672)\t1.0\n (0, 1676..."
1670,"(0, 225)\t1.0\n (0, 1672)\t1.0\n (0, 1676)..."
